In [ ]:
from imageanalysis.making_stacks import *
dir = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/'
process_directory_h5(dir, output_file='different_sides.h5',sort_by='time')

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'

from imageanalysis.calibrated_images_class import CalibratedImages
calibrated_images = CalibratedImages(different_sides_irradiation, font_path=font_path)

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'
from imageanalysis.features_analysis_class import FeaturesAnalysis

features_analysis = FeaturesAnalysis(different_sides_irradiation, font_path=font_path)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

datafile = 'Sample_2476/cluster_density_analysis/4l_gr.csv'
# datafile = 'analysis_results.csv'
df = pd.read_csv(datafile)
df['Histogram_Peaks']
df